In [1]:
import boto3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import precision_recall_cutoff as prc
from scipy.stats import boxcox
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.metrics import classification_report

# Defining the s3 bucket
s3 = boto3.resource('s3')
bucket_name = 'data-445-wagner'
bucket = s3.Bucket(bucket_name)

# Defining the file to be read from s3 bucket
file_key = "turnover.csv"

bucket_object = bucket.Object(file_key)
file_object = bucket_object.get()
file_content_stream = file_object.get('Body')

# Reading the csv file
turnover = pd.read_csv(file_content_stream)
turnover.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary
0,0.38,0.53,2,157,3,0,1,0,sales,low
1,0.80,0.86,5,262,6,0,1,0,sales,medium
2,0.11,0.88,7,272,4,0,1,0,sales,medium
3,0.72,0.87,5,223,5,0,1,0,sales,low
4,0.37,0.52,2,159,3,0,1,0,sales,low


In [2]:
#Changing sales and salary to dummies
turnover = pd.concat([turnover.drop(columns = ['sales']), pd.get_dummies(turnover['sales'])], axis = 1)
turnover = pd.concat([turnover, pd.get_dummies(turnover['salary'])], axis = 1)
turnover = turnover.rename(columns = {'low': 'Low', 'medium': 'Medium', 'high': 'High'})

In [4]:
#Using Box-Cox
transformed_time_spend, best_lambda = boxcox(turnover['time_spend_company'])
turnover['time_spend_company_bc'] = transformed_time_spend

#Transforming to 0-1 scale
scaler = MinMaxScaler(feature_range = (0,1))
turnover[['number_project', 'average_montly_hours']] = scaler.fit_transform(turnover[['number_project', 'average_montly_hours']])
turnover.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,salary,IT,...,management,marketing,product_mng,sales,support,technical,High,Low,Medium,time_spend_company_bc
0,0.38,0.53,0.0,0.285047,3,0,1,0,low,0,...,0,0,0,1,0,0,0,1,0,0.804651
1,0.80,0.86,0.6,0.775701,6,0,1,0,medium,0,...,0,0,0,1,0,0,0,0,1,1.098118
2,0.11,0.88,1.0,0.822430,4,0,1,0,medium,0,...,0,0,0,1,0,0,0,0,1,0.941381
3,0.72,0.87,0.6,0.593458,5,0,1,0,low,0,...,0,0,0,1,0,0,0,1,0,1.032330
4,0.37,0.52,0.0,0.294393,3,0,1,0,low,0,...,0,0,0,1,0,0,0,1,0,0.804651


In [5]:
#creating interactions
turnover['Interaction_1'] = turnover['satisfaction_level'] * turnover['time_spend_company']
turnover['Interaction_2'] = turnover['last_evaluation'] * turnover['promotion_last_5years']

In [6]:
#Defining Varibales
X = turnover.drop(columns = ['time_spend_company', 'salary', 'left'])
Y = turnover['left']

#Splitting data
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, stratify = Y)

In [8]:
#Building random forest model  
RF = RandomForestClassifier(n_estimators = 500, max_depth = 3).fit(X_train, Y_train)

#Predicting test
RF_pred = RF.predict_proba(X_test)[:, 1]

#Predicting labels
RF_labels =prc.precision_recall_cutoff(Y_test, RF_pred)

#Computing classification report
print(classification_report(Y_test, RF_labels))

              precision    recall  f1-score   support

           0       0.96      0.93      0.94      2286
           1       0.79      0.88      0.83       714

    accuracy                           0.92      3000
   macro avg       0.87      0.90      0.89      3000
weighted avg       0.92      0.92      0.92      3000



In [9]:
#Using train in new data frame for random forrest model 
X_train = X_train.drop(columns = ['Interaction_1', 'Interaction_2'])
X_test = X_test.drop(columns = ['Interaction_1', 'Interaction_2'])

#Building random forest model 2
RF = RandomForestClassifier(n_estimators = 500, max_depth = 3).fit(X_train, Y_train)

#Predicting test
RF_pred = RF.predict_proba(X_test)[:, 1]

#Predicting labels
RF_labels =prc.precision_recall_cutoff(Y_test, RF_pred)

#Computing classification report
print(classification_report(Y_test, RF_labels))

              precision    recall  f1-score   support

           0       0.91      0.98      0.94      2286
           1       0.92      0.69      0.79       714

    accuracy                           0.91      3000
   macro avg       0.92      0.84      0.87      3000
weighted avg       0.91      0.91      0.91      3000



In [ ]:
# I would use the model with two interactions to predict left